## Overview:

This notebook processes and analyzes model test result JSON files to extract unique model responses, their scores and frequencies, and save the results for different context types. It includes functions to:

- Load and process JSON files for a given context and ID.
- Extract unique responses, their scores, and frequencies.
- Save the processed data for all context types into a single JSON file for a specified ID.

Additionally, it demonstrates saving the results for a specific ID as an example.

### Helper Functions


Load and Process JSON Files:

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path

def __get_unique_model_responses(id: int, context_type: str, with_misleading: bool = False) -> list:
    """
    Load and process model test result JSON files for a given context and ID.

    Returns a list of unique model responses with their:
    - score (from first occurrence)
    - frequency (how many times that response appeared)
    
    Sorted by descending score.
    """
    model_name = 'llama-2-7b-80k'
    base_dir = Path('../results/graph')
    
    suffix = '_misleading' if with_misleading else ''
    directory = base_dir / f'{model_name}_id_{id}_{context_type}{suffix}'

    # Collect all responses and scores
    data = [
        json.load(open(directory / file, encoding='utf-8'))
        for file in os.listdir(directory) if file.endswith('.json')
    ]
    
    df = pd.DataFrame(data)[['model_response', 'score']]

    # Compute frequency
    freq = df['model_response'].value_counts().rename('frequency')

    # Drop duplicates but keep first score
    df = df.drop_duplicates(subset='model_response')

    # Merge with frequency
    df = df.merge(freq, left_on='model_response', right_index=True)

    # Sort by score descending
    df = df.sort_values(by='score', ascending=False).reset_index(drop=True)

    return df.to_dict(orient='records')


Save Processed Results:

In [ ]:
def save_all_contexts_for_id(id: int) -> None:
    """
    Gather results from all 4 context types and save them in a single JSON file for the given id.

    File will be saved as: unique_responses/{id}.json
    """
    save_dir = Path('unique_responses')
    save_dir.mkdir(parents=True, exist_ok=True)

    all_data = {
        'relevant': __get_unique_model_responses(id, 'relevant', with_misleading=False),
        'relevant_misleading': __get_unique_model_responses(id, 'relevant', with_misleading=True),
        'irrelevant': __get_unique_model_responses(id, 'irrelevant', with_misleading=False),
        'irrelevant_misleading': __get_unique_model_responses(id, 'irrelevant', with_misleading=True),
    }

    save_path = save_dir / f'{id}.json'
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print(f"Saved all context responses for ID {id} to {save_path}")

### Run Analysis:

In [33]:
save_all_contexts_for_id(160)

Saved all context responses for ID 160 to unique_responses/160.json
